# Mục mới

In [ ]:
import pandas as pd
import numpy as np
import nltk
#đọc data
df = pd.read_csv("https://raw.githubusercontent.com/huynhthanh98/AML/main/lab-04/spam_detection.csv")
df.head()

,Text,Label,y
0,"Go until jurong point, crazy.. Available only ...",ham,0
1,Ok lar... Joking wif u oni...,ham,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,1
3,U dun say so early hor... U c already then say...,ham,0
4,"Nah I don't think he goes to usf, he lives aro...",ham,0


In [ ]:
nltk.download('punkt')
texts = df["Text"].to_list()
texts = [text.lower() for text in texts ]           # chuyển các đoạn text thành chữ thường (word embedding chỉ cho chữ thường)
tokenized_texts = [nltk.tokenize.word_tokenize(text) for text in texts]    # tách câu thành một list các từ

print(tokenized_texts[0])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
['go', 'until', 'jurong', 'point', ',', 'crazy..', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', '...', 'cine', 'there', 'got', 'amore', 'wat', '...']


In [ ]:
#Tải bộ weights Glove
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2021-11-19 11:33:07--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-11-19 11:33:07--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-11-19 11:33:07--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
#Giải Nén
! unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
## không cần hiểu đống này lắm đâu
import io
import numpy as np
def load_word_embeddings(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    vocab, matrix = [], []
    i=0
    for line in fin:
        tokens = line.rstrip().split(' ')
        vocab.append(tokens[0])
        matrix.append(list(map(float, tokens[1:])))
    return vocab, np.asarray(matrix)

In [ ]:
#vocab là bộ từ điển các từ có trong Glove
#matrix chứa các vector biểu diễn các từ 
##Ta chọn bộ weighs có số chiều là 100
vocab, matrix = load_word_embeddings("glove.6B.100d.txt")

In [ ]:
#in thử shape của matrix
##Ta có thể thấy bộ từ điển này có 400000, mỗi từ là 1 vecotr có 100 chiều 
matrix.shape

(400000, 100)

In [ ]:
## Gán các mã
__PADDED_INDEX__ = 0    # mã dùng cho các vị trí chỉ có tính nối dài cho cùng kích thước
__UNKNOWN_WORD__ = 1    # mã cho những từ không có trong embedding

In [ ]:
# Tạo một dictionary, có nhiệm vụ là một ánh xạ từ ảnh sang mã số, mã số được bắt đầu từ 2 vì số 0 và 1 được dành cho trường hợp đặc biệt
word_to_index = {word: index+2 for index, word in enumerate(vocab)}

In [ ]:
# Do do mã số được bắt đầu từ 2, nên cần thêm 2 vector vào đàu ma trận
embedding_matrix = np.pad(matrix, ((2,0),(0,0)), mode='constant', constant_values =0.0)
print(embedding_matrix)

[[ 0.        0.        0.       ...  0.        0.        0.      ]
 [ 0.        0.        0.       ...  0.        0.        0.      ]
 [-0.038194 -0.24487   0.72812  ... -0.1459    0.8278    0.27062 ]
 ...
 [ 0.36088  -0.16919  -0.32704  ...  0.27139  -0.29188   0.16109 ]
 [-0.10461  -0.5047   -0.49331  ...  0.42527  -0.5125   -0.17054 ]
 [ 0.28365  -0.6263   -0.44351  ...  0.43678  -0.82607  -0.15701 ]]


In [ ]:
## Bây giờ ta sẽ chuyển data spam dection thành các mã số
import tensorflow as tf

X = []
for text in tokenized_texts:
    cur_text_indices = []
    for word in text:
        if word in word_to_index:
            cur_text_indices.append(word_to_index[word])    ## map từ word sang index
        else:
            cur_text_indices.append(__UNKNOWN_WORD__)       ## gán unknown cho từ không có trong bộ glove
    X.append(cur_text_indices)

## pad data cho có cùng chiều dài
X = tf.keras.preprocessing.sequence.pad_sequences(sequences=X,       # sequences: list các câu có độ dài không bằng nhau
                                                  padding='post')    # vị trí pad là 'pre' (trước) hoặc 'post' (sau)

y = df['y'].values   ## Label của bài toán, 0 là không phải spam, 1 là spam

In [ ]:
## Chia data
from sklearn.model_selection import train_test_split
X_1, X_test, y_1, y_test = train_test_split(X, y, test_size= 0.2, random_state =0)
X_train, X_valid, y_train, y_valid = train_test_split(X_1, y_1, test_size= 0.1, random_state =0)

# LSTM

In [ ]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense
from tensorflow.keras.models import Model

inputs = Input(shape=(None,))                   ## None biểu thị kích thước không xác định của câu

embed = Embedding(input_dim=embedding_matrix.shape[0],   ## Khai báo kích thước của vocab
                 output_dim=embedding_matrix.shape[1],   ## Khai báo kích thước của embedding
                  embeddings_initializer = tf.keras.initializers.Constant(value=embedding_matrix),  ## Khởi tạo cho embedding bằng ma trận có sẵn
                  trainable=False,                       ## Không cần thiết train embedding
                 mask_zero=True)(inputs)                 ## zero_mask: những vị trí có giá trị 0 không được tính toán, vì đó là giá trị thêm vào cho đủ độ dài mà thôi
                                                         ##  (__PADDED_INDEX__ gán bằng 0)

lstm = LSTM(units=100,                          ## units: kích thước của hidden_state trong LSTM
            return_sequences=False)(embed)      ## return_sequences: LSTM trả về toàn bộ  hay là trả về hidden_state cuối cùng

dense = Dense(units=2, activation='softmax')(lstm)
model = Model(inputs=inputs,
              outputs=dense)

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         40000200  
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 2)                 202       
                                                                 
Total params: 40,080,802
Trainable params: 80,602
Non-trainable params: 40,000,200
_________________________________________________________________


In [ ]:
#compile model
#model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=5e-5), loss=tf.keras.losses.sparse_categorical_crossentropy ,metrics=["accuracy"])

history=model.fit(x=X_train, y=y_train,
                  epochs=20,
                  verbose=1,
                  validation_data=(X_valid,y_valid))

#Xem độ chính xác trên tập test
evaluation=model.evaluate(X_test, y_test)
print("Model có độ chính xác là {} và loss là {}".format(evaluation[1], evaluation[0]))

Epoch 1/20
126/126 [==============================] - 14s 35ms/step - loss: 0.1608 - accuracy: 0.9387 - val_loss: 0.1085 - val_accuracy: 0.9731
Epoch 2/20
126/126 [==============================] - 3s 22ms/step - loss: 0.0772 - accuracy: 0.9728 - val_loss: 0.0782 - val_accuracy: 0.9821
Epoch 3/20
126/126 [==============================] - 3s 22ms/step - loss: 0.0550 - accuracy: 0.9813 - val_loss: 0.0859 - val_accuracy: 0.9821
Epoch 4/20
126/126 [==============================] - 3s 22ms/step - loss: 0.0499 - accuracy: 0.9836 - val_loss: 0.1343 - val_accuracy: 0.9507
Epoch 5/20
126/126 [==============================] - 3s 22ms/step - loss: 0.0457 - accuracy: 0.9870 - val_loss: 0.0762 - val_accuracy: 0.9821
Epoch 6/20
126/126 [==============================] - 3s 22ms/step - loss: 0.0352 - accuracy: 0.9905 - val_loss: 0.1063 - val_accuracy: 0.9776
Epoch 7/20
126/126 [==============================] - 3s 22ms/step - loss: 0.0296 - accuracy: 0.9910 - val_loss: 0.0811 - val_accuracy: 0.984

In [ ]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense, GRU, GlobalAveragePooling1D
from tensorflow.keras.models import Model

#inputs = Input(shape=(100,))
inputs = Input(shape=(215,))
#inputs = Input(shape=(None,))                   ## None biểu thị kích thước không xác định của câu

embed = Embedding(input_dim=embedding_matrix.shape[0],   ## Khai báo kích thước của vocab
                 output_dim=embedding_matrix.shape[1],   ## Khai báo kích thước của embedding
                  embeddings_initializer = tf.keras.initializers.Constant(value=embedding_matrix),  ## Khởi tạo cho embedding bằng ma trận có sẵn
                  trainable=False,                       ## Không cần thiết train embedding
                 mask_zero=True)(inputs)                 ## zero_mask: những vị trí có giá trị 0 không được tính toán, vì đó là giá trị thêm vào cho đủ độ dài mà thôi
                                                         ##  (__PADDED_INDEX__ gán bằng 0)

lstm = LSTM(units=100,                          ## units: kích thước của hidden_state trong LSTM
            return_sequences=True,
           return_state=True)(embed)      ## return_sequences: LSTM trả về toàn bộ  hay là trả về hidden_state cuối cùng
print(lstm)
global_maxpool = GlobalAveragePooling1D()(lstm[0])
dense = Dense(units=2, activation='softmax')(global_maxpool)
model1 = Model(inputs=inputs,
              outputs=dense)

model1.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model1.summary()

[<KerasTensor: shape=(None, 215, 100) dtype=float32 (created by layer 'lstm_1')>, <KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'lstm_1')>, <KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'lstm_1')>]
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 215)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 215, 100)          40000200  
                                                                 
 lstm_1 (LSTM)               [(None, 215, 100),        80400     
                              (None, 100),                       
                              (None, 100)]                       
                                                                 
 global_average_pooling1d (G  (None, 100)              0         
 lobalAveragePooling1D)

In [ ]:
#compile model
#model1.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=5e-5), loss=tf.keras.losses.sparse_categorical_crossentropy ,metrics=["accuracy"])

history=model1.fit(x=X_train, y=y_train,
                  epochs=20,
                  verbose=1,
                  validation_data=(X_valid,y_valid))

#Xem độ chính xác trên tập test
evaluation=model1.evaluate(X_test, y_test)
print("Model có độ chính xác là {} và loss là {}".format(evaluation[1], evaluation[0]))

Epoch 1/20
126/126 [==============================] - 9s 35ms/step - loss: 0.1682 - accuracy: 0.9367 - val_loss: 0.1766 - val_accuracy: 0.9462
Epoch 2/20
126/126 [==============================] - 3s 22ms/step - loss: 0.0888 - accuracy: 0.9713 - val_loss: 0.0776 - val_accuracy: 0.9731
Epoch 3/20
126/126 [==============================] - 3s 22ms/step - loss: 0.0653 - accuracy: 0.9786 - val_loss: 0.0701 - val_accuracy: 0.9753
Epoch 4/20
126/126 [==============================] - 3s 22ms/step - loss: 0.0616 - accuracy: 0.9796 - val_loss: 0.0703 - val_accuracy: 0.9753
Epoch 5/20
126/126 [==============================] - 3s 22ms/step - loss: 0.0472 - accuracy: 0.9850 - val_loss: 0.0632 - val_accuracy: 0.9843
Epoch 6/20
126/126 [==============================] - 3s 22ms/step - loss: 0.0377 - accuracy: 0.9893 - val_loss: 0.0570 - val_accuracy: 0.9776
Epoch 7/20
126/126 [==============================] - 3s 22ms/step - loss: 0.0339 - accuracy: 0.9888 - val_loss: 0.0619 - val_accuracy: 0.9843

# Bài tập

1. Tìm hiểu về cách dùng của GRU và chạy theo 2 cách hướng dẫn ở trên

In [ ]:
inputs = Input(shape=(None,))                   ## None biểu thị kích thước không xác định của câu

embed = Embedding(input_dim=embedding_matrix.shape[0],   ## Khai báo kích thước của vocab
                 output_dim=embedding_matrix.shape[1],   ## Khai báo kích thước của embedding
                  embeddings_initializer = tf.keras.initializers.Constant(value=embedding_matrix),  ## Khởi tạo cho embedding bằng ma trận có sẵn
                  trainable=False,                       ## Không cần thiết train embedding
                 mask_zero=True)(inputs)                 ## zero_mask: những vị trí có giá trị 0 không được tính toán, vì đó là giá trị thêm vào cho đủ độ dài mà thôi
                                                         ##  (__PADDED_INDEX__ gán bằng 0)

gru = GRU(units=100,                          ## units: kích thước của hidden_state trong LSTM
            return_sequences=False)(embed)      ## return_sequences: LSTM trả về toàn bộ  hay là trả về hidden_state cuối cùng

dense = Dense(units=2, activation='softmax')(gru)
model2 = Model(inputs=inputs,
              outputs=dense)

model2.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model2.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 100)         40000200  
                                                                 
 gru (GRU)                   (None, 100)               60600     
                                                                 
 dense_2 (Dense)             (None, 2)                 202       
                                                                 
Total params: 40,061,002
Trainable params: 60,802
Non-trainable params: 40,000,200
_________________________________________________________________


In [ ]:
#compile model
#model2.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=5e-5), loss=tf.keras.losses.sparse_categorical_crossentropy ,metrics=["accuracy"])

history=model2.fit(x=X_train, y=y_train,
                  epochs=20,
                  verbose=1,
                  validation_data=(X_valid,y_valid))

#Xem độ chính xác trên tập test
evaluation=model2.evaluate(X_test, y_test)
print("Model có độ chính xác là {} và loss là {}".format(evaluation[1], evaluation[0]))

Epoch 1/20
126/126 [==============================] - 9s 35ms/step - loss: 0.1991 - accuracy: 0.9228 - val_loss: 0.1376 - val_accuracy: 0.9529
Epoch 2/20
126/126 [==============================] - 2s 19ms/step - loss: 0.0826 - accuracy: 0.9728 - val_loss: 0.1104 - val_accuracy: 0.9664
Epoch 3/20
126/126 [==============================] - 3s 20ms/step - loss: 0.0517 - accuracy: 0.9855 - val_loss: 0.1219 - val_accuracy: 0.9641
Epoch 4/20
126/126 [==============================] - 2s 19ms/step - loss: 0.0406 - accuracy: 0.9888 - val_loss: 0.0769 - val_accuracy: 0.9798
Epoch 5/20
126/126 [==============================] - 3s 20ms/step - loss: 0.0361 - accuracy: 0.9898 - val_loss: 0.0924 - val_accuracy: 0.9776
Epoch 6/20
126/126 [==============================] - 3s 20ms/step - loss: 0.0261 - accuracy: 0.9933 - val_loss: 0.0823 - val_accuracy: 0.9798
Epoch 7/20
126/126 [==============================] - 2s 19ms/step - loss: 0.0178 - accuracy: 0.9963 - val_loss: 0.1095 - val_accuracy: 0.9753

In [ ]:
#inputs = Input(shape=(100,))
#inputs = Input(shape=(215,))
inputs = Input(shape=(None,))                   ## None biểu thị kích thước không xác định của câu

embed = Embedding(input_dim=embedding_matrix.shape[0],   ## Khai báo kích thước của vocab
                 output_dim=embedding_matrix.shape[1],   ## Khai báo kích thước của embedding
                  embeddings_initializer = tf.keras.initializers.Constant(value=embedding_matrix),  ## Khởi tạo cho embedding bằng ma trận có sẵn
                  trainable=False,                       ## Không cần thiết train embedding
                 mask_zero=True)(inputs)                 ## zero_mask: những vị trí có giá trị 0 không được tính toán, vì đó là giá trị thêm vào cho đủ độ dài mà thôi
                                                         ##  (__PADDED_INDEX__ gán bằng 0)

gru = GRU(units=100,                          ## units: kích thước của hidden_state trong LSTM
            return_sequences=True,
           return_state=True)(embed)      ## return_sequences: LSTM trả về toàn bộ  hay là trả về hidden_state cuối cùng
print(gru)
global_maxpool = GlobalAveragePooling1D()(gru[0])
dense = Dense(units=2, activation='softmax')(global_maxpool)
model3 = Model(inputs=inputs,
              outputs=dense)

model3.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model3.summary()

[<KerasTensor: shape=(None, None, 100) dtype=float32 (created by layer 'gru_1')>, <KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'gru_1')>]
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_3 (Embedding)     (None, None, 100)         40000200  
                                                                 
 gru_1 (GRU)                 [(None, None, 100),       60600     
                              (None, 100)]                       
                                                                 
 global_average_pooling1d_1   (None, 100)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None

In [ ]:
#compile model
model3.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=5e-5), loss=tf.keras.losses.sparse_categorical_crossentropy ,metrics=["accuracy"])

history=model3.fit(x=X_train, y=y_train,
                  epochs=20,
                  verbose=1,
                  validation_data=(X_valid,y_valid))

#Xem độ chính xác trên tập test
evaluation=model3.evaluate(X_test, y_test)
print("Model có độ chính xác là {} và loss là {}".format(evaluation[1], evaluation[0]))

Epoch 1/20
126/126 [==============================] - 8s 32ms/step - loss: 0.7890 - accuracy: 0.1635 - val_loss: 0.7663 - val_accuracy: 0.1771
Epoch 2/20
126/126 [==============================] - 3s 20ms/step - loss: 0.7391 - accuracy: 0.2567 - val_loss: 0.7191 - val_accuracy: 0.3498
Epoch 3/20
126/126 [==============================] - 2s 20ms/step - loss: 0.6956 - accuracy: 0.5019 - val_loss: 0.6780 - val_accuracy: 0.6300
Epoch 4/20
126/126 [==============================] - 2s 20ms/step - loss: 0.6578 - accuracy: 0.7493 - val_loss: 0.6423 - val_accuracy: 0.7960
Epoch 5/20
126/126 [==============================] - 3s 20ms/step - loss: 0.6249 - accuracy: 0.8303 - val_loss: 0.6112 - val_accuracy: 0.8296
Epoch 6/20
126/126 [==============================] - 3s 20ms/step - loss: 0.5963 - accuracy: 0.8557 - val_loss: 0.5841 - val_accuracy: 0.8520
Epoch 7/20
126/126 [==============================] - 3s 20ms/step - loss: 0.5713 - accuracy: 0.8592 - val_loss: 0.5604 - val_accuracy: 0.8543

In [ ]:
2. So sánh giữa 2 cấu trúc của GRU và LSTM. Từ đó đưa ra kết luận của mình (có dẫn chứng )

Theo như kết quả cho ra của LSTM và GRU (chạy theo 2 cách) thì ta thấy được GRU có số parameter nhiều hơn GRU (40,080,802 so với 40,061,002) nên tốc độ chạy của LSTM sẽ chậm hơn GRU. 
Về độ chính xác thì cách thứ 2 có độ chính xác cao hơn cách thứ nhất